# Maximum Likelihood Estimation

We will apply the maximum likelihood estimation to try to predict the angles theta and phi from the bloch sphere.

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
df = pd.read_csv("../../data/qst_mle_dataset.csv")
N_SHOTS = 200  # For each X, Y, Z basis
df.head()

,X_mean,Y_mean,Z_mean,theta_ideal,phi_ideal,cos_phi_ideal,sin_phi_ideal,X_ideal,Y_ideal,Z_ideal
0,0.93,-0.35,-0.30,1.824427,5.973514,0.952434,-0.304745,0.921963,-0.294996,-0.250920
1,0.72,0.50,-0.65,2.273101,0.557334,0.848668,0.528926,0.647835,0.403758,-0.645979
2,0.14,-1.00,-0.01,1.593512,4.840613,0.127873,-0.991791,0.127840,-0.991535,-0.022714
3,0.34,0.87,0.41,1.242172,1.163618,0.396020,0.918242,0.374828,0.869104,0.322741
4,-0.22,0.12,0.99,0.219199,2.852576,-0.958525,0.285009,-0.208429,0.061975,0.976072


In [4]:
# Compute probabilities based on the mean
df["probaX+"] = (1 + df["X_mean"]) / 2.0
df["probaY+"] = (1 + df["Y_mean"]) / 2.0
df["probaZ+"] = (1 + df["Z_mean"]) / 2.0

# Compute number of occurence of each state
# ! We take N_SHOTS = 200 as established before (dataset creation)
df["numberX"] = (df["probaX+"] * N_SHOTS).round().astype(int)
df["numberY"] = (df["probaY+"] * N_SHOTS).round().astype(int)
df["numberZ"] = (df["probaZ+"] * N_SHOTS).round().astype(int)

def angles_from_bloch(nx, ny, nz):
    """
    Given Bloch vector (nx, ny, nz) on the sphere, return (theta, phi).
    """
    # First we normalize (just in case)
    norm = np.sqrt(nx ** 2 + ny ** 2 + nz ** 2)
    nx /= norm
    ny /= norm
    nz /= norm
    
    # Calculate theta and phi
    theta = np.arccos(nz)
    phi = np.arctan2(ny,nx)
    # Keep phi between 0 and 2pi
    if phi < 0:
        phi += 2.0 * np.pi
    return theta, phi

def bloch_from_angles(theta, phi):
    """
    Given (theta, phi), return Bloch vector (nx, ny, nz).
    """
    # Just the normal formula to get the Bloch vector from the angles
    nx = np.sin(theta) * np.cos(phi)
    ny = np.sin(theta) * np.sin(phi)
    nz = np.cos(theta)
    return nx, ny, nz

def neg_log_likelihood(params, row):
    theta, phi = params # Parameters to optimize
    
    # Get the components of the estimated Bloch vector based on theta and phi (variables)
    nx, ny, nz = bloch_from_angles(theta, phi) 
    
    # New estimated probabilities 
    px_plus = (1.0 + nx) / 2.0
    py_plus = (1.0 + ny) / 2.0
    pz_plus = (1.0 + nz) / 2.0
    
    # Just to be sure to avoid log(0)
    eps = 1e-12
    px_plus = np.clip(px_plus, eps, 1.0 - eps)
    py_plus = np.clip(py_plus, eps, 1.0 - eps)
    pz_plus = np.clip(pz_plus, eps, 1.0 - eps)
    
    # Negative log likelihood formula
    nllx = -(df.iloc[row]["numberX"] * np.log(px_plus) + 
             (N_SHOTS - df.iloc[row]["numberX"]) * np.log(1 - px_plus))
    nlly = -(df.iloc[row]["numberY"] * np.log(py_plus) + 
             (N_SHOTS - df.iloc[row]["numberY"]) * np.log(1 - py_plus))
    nllz = -(df.iloc[row]["numberZ"] * np.log(pz_plus) + 
             (N_SHOTS - df.iloc[row]["numberZ"]) * np.log(1 - pz_plus))
    
    return nllx + nlly + nllz

def mle_row(row):
    # First estimation of theta and phi (based on the mean)
    theta0, phi0 = angles_from_bloch(df.iloc[row]["X_mean"], df.iloc[row]["Y_mean"], df.iloc[row]["Z_mean"])
    
    # Keep theta between 0 and pi and phi between 0 and 2pi
    bounds = [(0.0, np.pi), (0.0, 2.0 * np.pi)]
    
    # Minimize the Negative Log Likelihood
    res = minimize(
        fun=neg_log_likelihood,
        x0=np.array([theta0, phi0], dtype=float),
        args=(row,),
        bounds=bounds,
        method="L-BFGS-B"
    )
    
    theta_hat, phi_hat = res.x
    # In case phi = 2pi we bring it back to 0
    phi_hat = phi_hat % (2.0 * np.pi)
    
    return theta_hat, phi_hat

df.head()

,X_mean,Y_mean,Z_mean,theta_ideal,phi_ideal,cos_phi_ideal,sin_phi_ideal,X_ideal,Y_ideal,Z_ideal,probaX+,probaY+,probaZ+,numberX,numberY,numberZ
0,0.93,-0.35,-0.30,1.824427,5.973514,0.952434,-0.304745,0.921963,-0.294996,-0.250920,0.965,0.325,0.350,193,65,70
1,0.72,0.50,-0.65,2.273101,0.557334,0.848668,0.528926,0.647835,0.403758,-0.645979,0.860,0.750,0.175,172,150,35
2,0.14,-1.00,-0.01,1.593512,4.840613,0.127873,-0.991791,0.127840,-0.991535,-0.022714,0.570,0.000,0.495,114,0,99
3,0.34,0.87,0.41,1.242172,1.163618,0.396020,0.918242,0.374828,0.869104,0.322741,0.670,0.935,0.705,134,187,141
4,-0.22,0.12,0.99,0.219199,2.852576,-0.958525,0.285009,-0.208429,0.061975,0.976072,0.390,0.560,0.995,78,112,199


In [5]:
theta_hats = []
phi_hats = []

# Apply the algorithm to every state
for i in range(len(df)):
    theta_hat, phi_hat = mle_row(i)
    theta_hats.append(theta_hat)
    phi_hats.append(phi_hat)

df["theta_hat"] = theta_hats
df["phi_hat"] = phi_hats

theta_true = df["theta_ideal"].values
phi_true = df["phi_ideal"].values

theta_hat_arr = df["theta_hat"].values
phi_hat_arr = df["phi_hat"].values

# Compute the angle error of theta
df["theta_error"] = np.abs(theta_hat_arr - theta_true)

# Compute the angle error of phi (we must use this formula to ensure the right error as 2pi and 0 are the same)
phi_diff = (phi_hat_arr - phi_true + np.pi) % (2.0 * np.pi) - np.pi
df["phi_error"] = phi_diff

nx_true, ny_true, nz_true = bloch_from_angles(theta_true, phi_true)
nx_hat, ny_hat, nz_hat = bloch_from_angles(theta_hat_arr, phi_hat_arr)

# Computing fidelity according to the formula
dot_prod = nx_true * nx_hat + ny_true * ny_hat + nz_true * nz_hat
df["fidelity"] = np.abs(dot_prod) ** 2

print("Mean |theta_error|:", np.mean(np.abs(df["theta_error"])))
print("Mean |phi_error|  :", np.mean(np.abs(df["phi_error"])))
print("Mean fidelity     :", np.mean(df["fidelity"]))

df.head()

df.to_csv("results_mle.csv", index=False)

Mean |theta_error|: 0.04611649087611018
Mean |phi_error|  : 0.08107639120037077
Mean fidelity     : 0.9926145616104234
